In [1]:
import torch
import torchvision.transforms as transforms

import cfg
from utils import *

# set your own configs
args = cfg.parse_args()
print(args)
args.dataset = "REFUGE"
args.encoder = "vit_b"
GPUdevice = torch.device("mps", args.gpu_device)

# load the original SAM model 
args.sam_ckpt = sam_weights = 'checkpoint/sam/sam_vit_b_01ec64.pth'    # load the original SAM weight
net = get_network(args, args.net, use_gpu=args.gpu, gpu_device=GPUdevice, distribution = args.distributed)
net.eval()

with open(sam_weights, "rb") as f:
    state_dict = torch.load(f)
    new_state_dict = {k: v for k, v in state_dict.items() if k in net.state_dict() and net.state_dict()[k].shape == v.shape}
    net.load_state_dict(new_state_dict, strict = False)
print("loaded sam")

# load task-specific adapter
adapter_path = 'OpticDisc_Fundus_SAM_1024.pth'
checkpoint_file = os.path.join(adapter_path)
assert os.path.exists(checkpoint_file)
loc = 'mps:{}'.format(args.gpu_device)
checkpoint = torch.load(checkpoint_file, map_location=loc)
print("here")

state_dict = checkpoint['state_dict']
print(state_dict.keys())
print(net.state_dict().keys())
if args.distributed != 'none':
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        # name = k[7:] # remove `module.`
        name = 'module.' + k
        new_state_dict[name] = v
    # load params
else:
    new_state_dict = state_dict

net.load_state_dict(new_state_dict, strict = False)


/Users/i/miniconda3/envs/sam_adapt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(net='sam', baseline='unet', encoder='default', seg_net='transunet', mod='sam_adpt', exp_name='msa_test_isic', type='map', vis=5, reverse=False, pretrain=False, val_freq=5, gpu=True, gpu_device=0, sim_gpu=0, epoch_ini=1, image_size=256, out_size=256, patch_size=2, dim=512, depth=1, heads=16, mlp_dim=1024, w=4, b=2, s=True, warm=1, lr=0.0001, uinch=1, imp_lr=0.0003, weights=0, base_weights=0, sim_weights=0, distributed='none', dataset='isic', sam_ckpt=None, thd=False, chunk=None, num_sample=4, roi_size=96, evl_chunk=None, mid_dim=None, multimask_output=1, data_path='../data')
loaded sam
here
dict_keys(['image_encoder.blocks.0.MLP_Adapter.D_fc1.weight', 'image_encoder.blocks.0.MLP_Adapter.D_fc1.bias', 'image_encoder.blocks.0.MLP_Adapter.D_fc2.weight', 'image_encoder.blocks.0.MLP_Adapter.D_fc2.bias', 'image_encoder.blocks.0.Space_Adapter.D_fc1.weight', 'image_encoder.blocks.0.Space_Adapter.D_fc1.bias', 'image_encoder.blocks.0.Space_Adapter.D_fc2.weight', 'image_encoder.blocks.0.S

_IncompatibleKeys(missing_keys=['image_encoder.pos_embed', 'image_encoder.patch_embed.proj.weight', 'image_encoder.patch_embed.proj.bias', 'image_encoder.blocks.0.norm1.weight', 'image_encoder.blocks.0.norm1.bias', 'image_encoder.blocks.0.attn.rel_h', 'image_encoder.blocks.0.attn.rel_w', 'image_encoder.blocks.0.attn.qkv.weight', 'image_encoder.blocks.0.attn.qkv.bias', 'image_encoder.blocks.0.attn.proj.weight', 'image_encoder.blocks.0.attn.proj.bias', 'image_encoder.blocks.0.norm2.weight', 'image_encoder.blocks.0.norm2.bias', 'image_encoder.blocks.0.mlp.lin1.weight', 'image_encoder.blocks.0.mlp.lin1.bias', 'image_encoder.blocks.0.mlp.lin2.weight', 'image_encoder.blocks.0.mlp.lin2.bias', 'image_encoder.blocks.1.norm1.weight', 'image_encoder.blocks.1.norm1.bias', 'image_encoder.blocks.1.attn.rel_h', 'image_encoder.blocks.1.attn.rel_w', 'image_encoder.blocks.1.attn.qkv.weight', 'image_encoder.blocks.1.attn.qkv.bias', 'image_encoder.blocks.1.attn.proj.weight', 'image_encoder.blocks.1.attn.p